<a href="https://colab.research.google.com/github/santhoshsrivi/study/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! mkdir ~/.kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alagusantosh","key":"134ca1776e74692493140f151cb2885f"}'}

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d prajitdatta/movielens-100k-dataset

 63% 3.00M/4.77M [00:00<00:00, 5.60MB/s]
100% 4.77M/4.77M [00:00<00:00, 6.93MB/s]


In [8]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811629 sha256=0d105fb97edea1ca13796a9c14b7ad9780a084d182469a2c626a2e73220a6cd5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [9]:
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import NormalPredictor, SVD

import numpy as np
import pandas as pd
import re

In [10]:
!unzip movielens-100k-dataset.zip

Archive:  movielens-100k-dataset.zip
  inflating: ml-100k/README          
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [12]:
import pandas as pd

In [16]:
user_columns = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users_df = pd.read_csv('ml-100k/u.user',sep='|',names=user_columns)

In [17]:
users_df.head(5)

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [19]:
ratings_columns = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_df = pd.read_csv('ml-100k/u.data',sep='\t',names=ratings_columns)
ratings_df.drop( "unix_timestamp", inplace = True, axis = 1 )
ratings_df.head(5)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


**Read and clean movie data**

In [20]:
def clean_title(title):
    return re.sub("[\(\[].*?[\)\]]", "",title)

def process_genre(series):
    genres = series.index[6:-2]

    text = []
    for i in genres:
        if series[i] == 1:
            text.append(i)
            break
    return ", ".join(text)

In [22]:
genre_df = pd.read_csv('ml-100k/u.genre', sep='|', encoding='latin-1')
genre_columns = ["unknown"] + list(genre_df[genre_df.columns[0]].values)
print(genre_columns)

['unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [23]:
movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies_df = pd.read_csv('ml-100k/u.item', sep='|', names=movie_columns+genre_columns,
                     encoding='latin-1')

In [24]:
movies_df.head(5)

,movie_id,title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Removing the issuing date from the movie title and created a copy of the movie dataset where we concatenate the genres, so that we have a list with the genres, and we eliminate the separate genre columns.



In [26]:
import re

In [27]:
movies_df['title'] = movies_df['title'].apply(clean_title)
movies_df['title'] = movies_df['title'].str.strip()

movies_df['genre'] = movies_df.apply(process_genre,axis=1)

In [29]:
movies_df.head(2)

,movie_id,title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,1,Toy Story,01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,Animation
1,2,GoldenEye,01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action


In [30]:
movies_red_df = movies_df.copy()
movies_red_df.drop(movies_red_df.columns[[2,3,4]], inplace = True, axis = 1)
movies_red_df.drop(genre_columns,axis=1,inplace=True)

movies_red_df.head(5)

,movie_id,title,genre
0,1,Toy Story,Animation
1,2,GoldenEye,Action
2,3,Four Rooms,Thriller
3,4,Get Shorty,Action
4,5,Copycat,Crime


In [31]:
ratings_df.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [32]:
ratings_df.user_id.nunique()

943

In [33]:
reader = Reader(rating_scale=(1, 5))

In [34]:
data = Dataset.load_from_df(ratings_df, reader=reader)

**Model with SVD**

*if you have a dataset of movie ratings, and you want to use the SVD object to build a recommender system, you would need to set the n_factors parameter to the number of latent factors that you think are necessary to represent the data.*

In [35]:
algo_svd = SVD(n_factors=40)

In [36]:
cross_validate(algo_svd, data, measures = ["RMSE", "MAE", "FCP"], cv=5, verbose=True)

Evaluating RMSE, MAE, FCP of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9284  0.9407  0.9407  0.9334  0.9334  0.9353  0.0048  
MAE (testset)     0.7294  0.7403  0.7461  0.7374  0.7344  0.7375  0.0056  
FCP (testset)     0.7011  0.6981  0.7026  0.7003  0.6991  0.7002  0.0016  
Fit time          3.18    1.00    1.02    1.01    1.01    1.44    0.87    
Test time         0.60    0.13    0.14    0.13    0.25    0.25    0.18    


{'test_rmse': array([0.92839655, 0.9406917 , 0.94070158, 0.93339373, 0.93344502]),
 'test_mae': array([0.72940474, 0.74034095, 0.74608226, 0.7373687 , 0.73442005]),
 'test_fcp': array([0.701079  , 0.69813467, 0.70263776, 0.70025359, 0.69909182]),
 'fit_time': (3.1809182167053223,
  0.9968914985656738,
  1.015488624572754,
  1.0079712867736816,
  1.0085904598236084),
 'test_time': (0.595282793045044,
  0.13109493255615234,
  0.1405172348022461,
  0.12573671340942383,
  0.25464606285095215)}